## Preprocessing

In [281]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [282]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(columns="EIN")
application_df=application_df.drop(columns="NAME")
application_df=application_df.drop(columns="SPECIAL_CONSIDERATIONS")

In [283]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE

In [284]:
application_df.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [285]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE

In [286]:
print(application_df.value_counts(["APPLICATION_TYPE"], sort=True))


APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
T9                    156
T13                    66
T12                    27
T2                     16
T14                     3
T25                     3
T29                     2
T15                     2
T17                     1
dtype: int64


In [287]:

print(application_df.value_counts(["ASK_AMT"], sort=True))

ASK_AMT   
5000          25398
63981             3
10478             3
15583             3
6725              3
              ...  
53268             1
53251             1
53248             1
53241             1
8597806340        1
Length: 8747, dtype: int64


In [288]:
#application_df=application_df[application_df.groupby('ASK_AMT').ASK_AMT.transform('count')>10].copy() 
application_df['ASK_AMT'].value_counts(bins = 100, sort=False)



(-8592801.341, 85983013.4]      34186
(85983013.4, 171961026.8]          33
(171961026.8, 257939040.2]         24
(257939040.2, 343917053.6]          9
(343917053.6, 429895067.0]          5
                                ...  
(8167916273.0, 8253894286.4]        0
(8253894286.4, 8339872299.8]        0
(8339872299.8, 8425850313.2]        0
(8425850313.2, 8511828326.6]        0
(8511828326.6, 8597806340.0]        2
Name: ASK_AMT, Length: 100, dtype: int64

In [289]:
print(application_df.value_counts(["ASK_AMT"], sort=True))

ASK_AMT   
5000          25398
63981             3
10478             3
15583             3
6725              3
              ...  
53268             1
53251             1
53248             1
53241             1
8597806340        1
Length: 8747, dtype: int64


In [290]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE

import numpy as np
application_types_to_replace=application_df.APPLICATION_TYPE.value_counts()
val=application_types_to_replace[:8].index
application_df['APPLICATION_TYPE'] = application_df.APPLICATION_TYPE.where(application_df.APPLICATION_TYPE.isin(val), 'Other')
print(application_df.value_counts(["APPLICATION_TYPE"], sort=True))


APPLICATION_TYPE
T3                  27037
T4                   1542
T6                   1216
T5                   1173
T19                  1065
T8                    737
T7                    725
T10                   528
Other                 276
dtype: int64


In [291]:
application_df=application_df[application_df.groupby('CLASSIFICATION').CLASSIFICATION.transform('count')>1].copy() 

In [292]:
print(application_df.value_counts(["CLASSIFICATION"], sort=True))

CLASSIFICATION
C1000             17326
C2000              6074
C1200              4837
C3000              1918
C2100              1883
C7000               777
C1700               287
C4000               194
C5000               116
C1270               114
C2700               104
C2800                95
C7100                75
C1300                58
C1280                50
C1230                36
C1400                34
C7200                32
C2300                32
C1240                30
C8000                20
C7120                18
C1500                16
C6000                15
C1800                15
C1250                14
C8200                11
C1278                10
C1238                10
C1235                 9
C1237                 9
C7210                 7
C1720                 6
C4100                 6
C2400                 6
C1257                 5
C1600                 5
C1260                 3
C2710                 3
C0                    3
C1256                 2
C

In [293]:
# Look at CLASSIFICATION value counts for binning
#  YOUR CODE GOES HERE
application_df = application_df[application_df.groupby('CLASSIFICATION').CLASSIFICATION.transform('count')>1].copy() 
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [294]:
# Convert categorical data to numeric with `pd.get_dummies`
#  YOUR CODE GOES HERE
application_df=pd.get_dummies(application_df)
application_df.dtypes

STATUS                    int64
ASK_AMT                   int64
IS_SUCCESSFUL             int64
APPLICATION_TYPE_Other    uint8
APPLICATION_TYPE_T10      uint8
                          ...  
INCOME_AMT_10M-50M        uint8
INCOME_AMT_1M-5M          uint8
INCOME_AMT_25000-99999    uint8
INCOME_AMT_50M+           uint8
INCOME_AMT_5M-10M         uint8
Length: 81, dtype: object

In [295]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE

X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [296]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [297]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

# Compile the Sequential model together and customize metrics



# First hidden layer
#  YOUR CODE GOES HERE

# Second hidden layer
#  YOUR CODE GOES HERE

# Output layer
#  YOUR CODE GOES HERE

# Check the structure of the model


In [298]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE
nn_model = tf.keras.models.Sequential()
# Compile the Sequential model together and customize metrics



# First hidden layer
#  YOUR CODE GOES HERE
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=80))

# Second hidden layer
#  YOUR CODE GOES HERE

nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))
# Output layer
#  YOUR CODE GOES HERE
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the modelnn_model = tf.keras.models.Sequential()


nn_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 80)                6480      
                                                                 
 dense_31 (Dense)            (None, 30)                2430      
                                                                 
 dense_32 (Dense)            (None, 1)                 31        
                                                                 
Total params: 8,941
Trainable params: 8,941
Non-trainable params: 0
_________________________________________________________________


In [299]:
# Compile the model
#  YOUR CODE GOES HERE
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [300]:
#Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5697 - accuracy: 0.7227
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7302
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7332
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7340
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5446 - accuracy: 0.7345
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5434 - accuracy: 0.7359
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5421 - accuracy: 0.7366
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5408 - accuracy: 0.7363
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7379
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5399 - accuracy: 0.7375

In [301]:
# Evaluate the model using the test data
# model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
# print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [302]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5716 - accuracy: 0.7248 - 387ms/epoch - 1ms/step
Loss: 0.5716357827186584, Accuracy: 0.7248220443725586


ASK AMOUNT is only fixed at 5000 amount
199/199 - 0s - loss: 0.5327 - accuracy: 0.7514 - 433ms/epoch - 2ms/step
Loss: 0.5327332615852356, Accuracy: 0.7514186501502991

Without ASK amount limit to 5000,
268/268 - 1s - loss: 0.5719 - accuracy: 0.7232 - 500ms/epoch - 2ms/step
Loss: 0.571903645992279, Accuracy: 0.7231882214546204

With 1000 bins for ASK AMOUNT:
268/268 - 0s - loss: 0.5697 - accuracy: 0.7217 - 376ms/epoch - 1ms/step
Loss: 0.5697351694107056, Accuracy: 0.7216711640357971

With 100 bins for ASK_AMOUNT

268/268 - 0s - loss: 0.5716 - accuracy: 0.7248 - 387ms/epoch - 1ms/step
Loss: 0.5716357827186584, Accuracy: 0.7248220443725586

Anser sheet
8575/8575 - 0s - loss: 0.5578 - acc: 0.7263
Loss: 0.557812534073699, Accuracy: 0.7262973785400391

In [58]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn_model.save("nn_model.h5")